In [ ]:
import sys

sys.path.append("../..")
from config import set_project_root

set_project_root(levels_up=3)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from research.datasets import CRSP
from research.utils import summary

In [ ]:
# Read in crsp monthly data

df = CRSP().df.copy()

df.head()

In [ ]:
# In sample
start = np.datetime64("1929-01-01")
end = np.datetime64("1982-12-31")

df = df[(df["date"] >= start) & (df["date"] <= end)]

df = df.reset_index(drop=True)

df

In [ ]:
# Cleaning

df = df[["permno", "ticker", "date", "prc", "ret"]].copy()

df["mdt"] = pd.to_datetime(df["date"]).dt.strftime("%Y-%m")
df["month"] = pd.to_datetime(df["date"]).dt.strftime("%m")

df.head()

In [ ]:
# Calculate Reversal feature
window = 1

# Log Returns
df["logret"] = np.log1p(df["ret"])

# Reversal from t-1 to t (also know as 1 month momentum)
df["rev_1"] = df.groupby("permno")["logret"].rolling(1, 1).sum().reset_index(drop=True)
df["rev_1"] = df.groupby("permno")["rev_1"].shift(1)

# Reversal from t-12 to t (also know as 1 month momentum)
df["rev_12"] = (
    df.groupby("permno")["logret"].rolling(12, 12).sum().reset_index(drop=True)
)
df["rev_12"] = df.groupby("permno")["rev_12"].shift(1)

df

In [ ]:
# Filtering by prc greater than 5 and months that momentum is known

df["prclag"] = df.groupby("permno")["prc"].shift(1)

df = df.query("rev_1 == rev_1 and rev_12 == rev_12 and prclag >= 5").reset_index(
    drop=True
)

df

In [ ]:
# Reversal decile bins
df["rev_1_bins"] = df.groupby("mdt")["rev_1"].transform(
    lambda x: pd.qcut(x, 10, labels=False, duplicates="drop")
)  # I would like to not drop dulplicates
df["rev_12_bins"] = df.groupby("mdt")["rev_12"].transform(
    lambda x: pd.qcut(x, 10, labels=False, duplicates="drop")
)  # I would like to not drop dulplicates

df

In [ ]:
# Form portfolios on reversal bins

# Portfolio df for summary stats
port_s1 = (
    df.groupby(["mdt", "rev_1_bins"])["ret"].mean().unstack(level=["rev_1_bins"]) * 100
)
port_s12 = (
    df.groupby(["mdt", "rev_12_bins"])["ret"].mean().unstack(level=["rev_12_bins"])
    * 100
)

# Drop rows from port where we don't know the holding return
port_s1 = port_s1.dropna()
port_s12 = port_s12.dropna()

# Spread portfolio: Long loser. Short winner.
port_s1["spread"] = port_s1[0] - port_s1[9]
port_s12["spread"] = port_s12[9] - port_s12[0]

display(port_s1, port_s12)

In [ ]:
from tabulate import tabulate


def table(data: pd.DataFrame, title: str = ""):
    return title + "\n" + tabulate(data, headers="keys", tablefmt="heavy_grid") + "\n"

In [ ]:
port_s1

In [ ]:
summary(port_s1).round(3)

In [ ]:
# Summary

rev_1_summary = summary(port_s1).loc[["mean", "std", "tstat"]].round(3)
rev_12_summary = summary(port_s12).loc[["mean", "std", "tstat"]].round(3)

print(table(data=rev_1_summary, title="1 Month Reversal Portfolios"))
print(table(data=rev_12_summary, title="12 Month Reversal Portfolio"))

In [ ]:
# Log return dataframe
log_ret = np.log1p(port_s1 / 100)

# Cummulative return dataframe
cum_ret = log_ret.cumsum()
cum_ret.index = pd.to_datetime(cum_ret.index)

# Lineplot
plt.figure(figsize=(10, 6))

for col in cum_ret.columns:
    color = "red" if col == "spread" else None
    plt.plot(cum_ret[col], label=f"Port {col}", color=color)

plt.title("S1 Reversal Portfolios (Rebalanced Monthly)")
plt.ylabel("Cummulative Monthly Returns")
plt.legend()
plt.show()

In [ ]:
# Log return dataframe
log_ret = np.log1p(port_s12 / 100)

# Cummulative return dataframe
cum_ret = log_ret.cumsum()
cum_ret.index = pd.to_datetime(cum_ret.index)

# Lineplot
plt.figure(figsize=(10, 6))

for col in cum_ret.columns:
    color = "red" if col == "spread" else None
    plt.plot(cum_ret[col], label=f"Port {col}", color=color)

plt.title("S12 Reversal Portfolios (Rebalanced Monthly)")
plt.ylabel("Cummulative Monthly Returns")
plt.legend()
plt.show()